In [1]:
from qiskit_nature.drivers import Molecule
from qiskit_nature.drivers.second_quantization import ElectronicStructureMoleculeDriver, ElectronicStructureDriverType
from qiskit_nature.mappers.second_quantization import ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.converters.second_quantization import QubitConverter
from qiskit_nature.problems.second_quantization.electronic import ElectronicStructureProblem
from qiskit_nature.transformers.second_quantization.electronic import ActiveSpaceTransformer
import warnings
from qiskit_nature.circuit.library import HartreeFock,UCCSD
from qiskit import Aer
from qiskit.algorithms.optimizers import COBYLA, L_BFGS_B, SPSA, SLSQP
from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
from qiskit_nature.algorithms.ground_state_solvers import GroundStateEigensolver
import numpy as np 
warnings.filterwarnings("ignore")


/tmp/ipykernel_4759/3598970827.py:11: NatureDeprecationWarning: The qiskit_nature.algorithms.excited_state_solvers package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release. Instead use the qiskit_nature.second_q.algorithms.excited_state_solvers package.
  from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory
/tmp/ipykernel_4759/3598970827.py:11: NatureDeprecationWarning: The qiskit_nature.algorithms.pes_sampler package is deprecated as of version 0.5.0 and will be removed no sooner than 3 months after the release . Please refer to 'https://github.com/Qiskit/qiskit-nature/issues/750' for additional information regarding 'extrapolators'.
  from qiskit_nature.algorithms.ground_state_solvers.minimum_eigensolver_factories import NumPyMinimumEigensolverFactory


In [4]:
molecule = Molecule(
    # coordinates are given in Angstrom
    geometry=[
        ["O", [0.0, 0.0, 0.0]],
        ["H", [0.758602, 0.0, 0.504284]],
        ["H", [0.758602, 0.0, -0.504284]]
    ],
    multiplicity=1,  # = 2*spin + 1
    charge=0,
)

driver = ElectronicStructureMoleculeDriver(
    molecule=molecule,
    basis="sto3g",
    driver_type=ElectronicStructureDriverType.PYSCF,
)

properties = driver.run()
converter = QubitConverter(JordanWignerMapper())
PN_property = properties.get_property("ParticleNumber")
print(PN_property)

# Define the active space around the Fermi level 
# (selected automatically around the HOMO and LUMO, ordered by energy)
transformer = ActiveSpaceTransformer(
    num_electrons=2, #how many electrons we have in our active space
    num_molecular_orbitals=2, #how many orbitals we have in our active space
)

problem = ElectronicStructureProblem(driver, [transformer])

second_q_ops = problem.second_q_ops() # this calls driver.run() internally

hamiltonian = second_q_ops['ElectronicEnergy']

print(hamiltonian)
qubit_op = converter.convert(hamiltonian)
print('转换后\n',qubit_op)

particle_number = problem.grouped_property_transformed.get_property("ParticleNumber")
num_spin_orbitals = particle_number.num_spin_orbitals
num_particles = particle_number.num_particles
init_state = HartreeFock(num_spin_orbitals, num_particles, converter)
print(init_state)

ParticleNumber:
	14 SOs
	5 alpha electrons
		orbital occupation: [1. 1. 1. 1. 1. 0. 0.]
	5 beta electrons
		orbital occupation: [1. 1. 1. 1. 1. 0. 0.]
Fermionic Operator
register length=4, number terms=36
  -1.2879605266732712 * ( +_0 -_0 )
+ -0.574996432518691 * ( +_1 -_1 )
+ -1.2879605266732712 * ( +_2 -_2 )
+ -0.574996432518691 * ( +_3 -_3 )
+ -0.44007954668752264 * ( +_0 +_0 -_0 -_0 )
+ -0.0192 ...
转换后
 -0.8720633572288583 * IIII
+ 0.11923749314031427 * IIIZ
- 0.17865261550743755 * IIZI
+ 0.11923749314031434 * IZII
- 0.17865261550743752 * ZIII
+ 0.14753433345685837 * IIZZ
+ 0.22003977334376132 * IZIZ
+ 0.009634329938843231 * YYYY
+ 0.009634329938843231 * XXYY
+ 0.009634329938843231 * YYXX
+ 0.009634329938843231 * XXXX
+ 0.15716866339570162 * ZIIZ
+ 0.15716866339570162 * IZZI
+ 0.16144783491422307 * ZIZI
+ 0.14753433345685837 * ZZII
     ┌───┐
q_0: ┤ X ├
     └───┘
q_1: ─────
     ┌───┐
q_2: ┤ X ├
     └───┘
q_3: ─────
          


In [13]:
uccsd = UCCSD(initial_state=init_state,num_particles=num_particles,num_spin_orbitals=num_spin_orbitals,qubit_converter=converter)
fop = uccsd.excitation_ops()
fsop = converter.convert(fop[0])
fsop

PauliSumOp(SparsePauliOp(['IIXY', 'IIYX'],
              coeffs=[ 0.5+0.j, -0.5+0.j]), coeff=1.0)

In [ ]:
ansatz = UCCSD(converter,num_particles,num_spin_orbitals,initial_state = init_state)
print(ansatz.decompose())
#后端设定
backend = Aer.get_backend('statevector_simulator')
optimizer = L_BFGS_B(maxfun=500)

     ┌───┐┌───────────────────────────────────┐»
q_0: ┤ X ├┤0                                  ├»
     └───┘│                                   │»
q_1: ─────┤1                                  ├»
     ┌───┐│  exp(-it (IIXY + IIYX))(1.0*t[0]) │»
q_2: ┤ X ├┤2                                  ├»
     └───┘│                                   │»
q_3: ─────┤3                                  ├»
          └───────────────────────────────────┘»
«     ┌───────────────────────────────────┐»
«q_0: ┤0                                  ├»
«     │                                   │»
«q_1: ┤1                                  ├»
«     │  exp(-it (XYII + YXII))(1.0*t[1]) │»
«q_2: ┤2                                  ├»
«     │                                   │»
«q_3: ┤3                                  ├»
«     └───────────────────────────────────┘»
«     ┌─────────────────────────────────────────────────────────────────────────────┐
«q_0: ┤0                                                            

In [ ]:
def exact_diagonalizer(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

result_exact = exact_diagonalizer(problem, converter)
exact_energy = np.real(result_exact.eigenenergies[0])
print("Exact electronic energy", exact_energy)
print(result_exact)

Exact electronic energy -1.697007028852889
=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -84.700895930366
  - computed part:      -1.697007028853
  - ActiveSpaceTransformer extracted energy part: -83.003888901513
~ Nuclear repulsion energy (Hartree): 9.819476509782
> Total ground state energy (Hartree): -74.881419420584
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 2.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [2.86710004  0.0  0.0]
 
  0: 
  * Electronic dipole moment (a.u.): [2.0649676  0.0  0.0]
    - computed part:      [0.0022825  0.0  0.0]
    - ActiveSpaceTransformer extracted energy part: [2.0626851  0.0  0.0]
  > Dipole moment (a.u.): [0.80213244  0.0  0.0]  Total: 0.80213244
                 (debye): [2.0388171  0.0  0.0]  Total: 2.0388171
 


In [4]:
import numpy as np
from qiskit.algorithms.optimizers import SLSQP
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.primitives import Estimator
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE
from qiskit_nature.second_q.algorithms import GroundStateEigensolver

new_ansatz = UCCSD(num_particles=num_particles,num_spin_orbitals=num_spin_orbitals,qubit_converter=converter,initial_state=init_state)
vqe = VQE(Estimator(), ansatz, SLSQP())
vqe.initial_point = np.zeros(ansatz.num_parameters)
adapt_vqe = AdaptVQE(vqe)
adapt_vqe.supports_aux_operators = lambda: True  # temporary fix
solver = GroundStateEigensolver(converter, adapt_vqe)
result = solver.solve(problem)




NameError: name 'num_particles' is not defined

In [ ]:
from AdaptVQE/